In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument('--headless')

driver = webdriver.Chrome(options=options)

In [2]:
import time
import pandas as pd
from io import StringIO

def scrape_url(year, week):
    driver.get(url=f"https://www.fantasypros.com/nfl/rankings/dst.php?year={year}&week={week}")
    time.sleep(5)  # give time for page to load

    data = pd.read_html(StringIO(driver.page_source))[0]
    data = data[['RK', 'Player Name', 'OPP']]

    # get rid of team from player name
    data['Player Name'] = data['Player Name'].str.split(' \(').str[0]

    data.insert(0, "Year", year)
    data.insert(1, "Week", week)

    return data

# scrape_url(2023, 1)

In [3]:
from itertools import product
from tqdm.autonotebook import tqdm

d = []
for year, week in tqdm(list(product([year for year in range(2018, 2024)], [week for week in range(1, 18)]))):
    if (week != 17) or (year >= 2021):
        d.append(scrape_url(year, week))

C:\Users\mhugh\AppData\Local\Temp\ipykernel_3312\1166229690.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/102 [00:00<?, ?it/s]

In [4]:
df = pd.concat(d, ignore_index=True)

df.insert(df.columns.get_loc('Player Name') + 1, 'POS', 'DST')

df.rename(columns={'RK': 'POS RK'}, inplace=True)

df

,Year,Week,RK,Player Name,OPP
0,2018,1,1,Baltimore Ravens,vs. BUF
1,2018,1,2,New Orleans Saints,vs. TB
2,2018,1,3,Jacksonville Jaguars,at NYG
3,2018,1,4,Los Angeles Rams,at LV
4,2018,1,5,Minnesota Vikings,vs. SF
...,...,...,...,...,...
2971,2023,17,28,Cincinnati Bengals,at KC
2972,2023,17,29,Detroit Lions,at DAL
2973,2023,17,30,New York Giants,vs. LAR
2974,2023,17,31,Arizona Cardinals,at PHI


In [6]:
df.groupby(['Year', 'Week'])['Player Name'].count().min()

np.int64(26)

In [7]:
df.to_parquet(r'..\..\data\fantasypros_in_season_rankings\defense_rankings.parquet')

'Done'

'Done'